In [4]:
import os
import pandas as pd
import numpy as np

import tensorflow as tf

In [5]:
train = pd.read_csv('data/train_features.csv')
train_labels = pd.read_csv('data/train_labels.csv')
test = pd.read_csv('data/test_features.csv')
submission = pd.read_csv('data/sample_submission.csv')

In [6]:
# id랑 time label을 없앰
X=tf.reshape(np.array(train.iloc[:,2:]),[-1, 600, 6])
X.shape

TensorShape([3125, 600, 6])

In [7]:
# label을 one-hot encoding (61개의 unique 한 label 존재)
y = tf.keras.utils.to_categorical(train_labels['label']) 
y.shape

(3125, 61)

In [8]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Activation, GRU
from keras import optimizers

# 층 없애고
# batch Normalization
# weight decay
def deep_gru():
    model = Sequential()
    model.add(GRU(32, input_shape = (600,6), return_sequences = True))
    model.add(GRU(64, return_sequences = True))
    model.add(GRU(64, return_sequences = True))   
    model.add(GRU(128, return_sequences = False))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(61, activation='softmax'))
    
    #adam = optimizers.adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])
    return model

In [9]:
model1 = deep_gru()
model1.fit(X,y, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
79/79 [==============================] - 146s 2s/step - loss: 3.0764 - accuracy: 0.4403 - val_loss: 2.7413 - val_accuracy: 0.4976
Epoch 2/100
79/79 [==============================] - 99s 1s/step - loss: 2.7386 - accuracy: 0.4623 - val_loss: 2.5491 - val_accuracy: 0.5024
Epoch 3/100
79/79 [==============================] - 102s 1s/step - loss: 2.4838 - accuracy: 0.4821 - val_loss: 2.4133 - val_accuracy: 0.5120
Epoch 4/100
79/79 [==============================] - 111s 1s/step - loss: 2.2642 - accuracy: 0.4985 - val_loss: 2.2698 - val_accuracy: 0.5232
Epoch 5/100
79/79 [==============================] - 108s 1s/step - loss: 2.0597 - accuracy: 0.5205 - val_loss: 2.2626 - val_accuracy: 0.5184
Epoch 6/100
79/79 [==============================] - 116s 1s/step - loss: 1.8894 - accuracy: 0.5379 - val_loss: 2.0190 - val_accuracy: 0.5456
Epoch 7/100
79/79 [==============================] - 120s 2s/step - loss: 1.7034 - accuracy: 0.5714 - val_loss: 2.3035 - val_accuracy: 0.5536
Epoch 8

KeyboardInterrupt: 

In [10]:
test_X=tf.reshape(np.array(test.iloc[:,2:]),[-1, 600, 6])
test_X.shape

TensorShape([782, 600, 6])

In [11]:
prediction=model1.predict(test_X)

In [12]:
submission.iloc[:,1:]=prediction

In [13]:
submission.to_csv('deepGRU_submission.csv', index=False)